In [6]:
# data analsysis
import pandas as pd 
# data in a vectorized manner manipulation
import numpy as np 
# tranform JSON file
from pandas.io.json import json_normalize 
# JSON files manipulation
import json 
# HTTP library
import requests 
# scraping library
from bs4 import BeautifulSoup 
# sklearn clustering algorithm
from sklearn.cluster import KMeans 
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

print('Libraries imported.')

Libraries imported.


In [43]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
website_url = requests.get(url).text
soup = BeautifulSoup(website_url,'lxml')
# print(soup.prettify())

In [44]:
My_table = soup.find('table',{'class':'wikitable sortable'})
table1=""
for tr in My_table.find_all('tr'):
    row1=""
    for tds in tr.find_all('td'):
        row1=row1+","+tds.text
    table1=table1+row1[1:]
#print(table1)

In [45]:
file = open("data/toronto.csv","wb")
file.write(bytes(table1,encoding="ascii",errors="ignore"))

df = pd.read_csv('data/toronto.csv',header=None)
df.columns=["Postalcode","Borough","Neighbourhood"]
print(df.shape)
df.head()

(287, 3)


,Postalcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [46]:
# Get names of indexes for which column Borough has value "Not assigned"
indexNames = df[ df['Borough'] =='Not assigned'].index
# Delete these row indexes from dataFrame
df.drop(indexNames , inplace=True) 
print(df.shape)
df.head()

(210, 3)


,Postalcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor


In [47]:
#a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough
df.loc[df['Neighbourhood'] =='Not assigned' , 'Neighbourhood'] = df['Borough']
print(df.shape)
df.head(12)

(210, 3)


,Postalcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
7,M7A,Queen's Park,Queen's Park
9,M9A,Downtown Toronto,Queen's Park
10,M1B,Scarborough,Rouge
11,M1B,Scarborough,Malvern
13,M3B,North York,Don Mills North


In [63]:
result = df.groupby(['Postalcode','Borough'], sort=False).agg( ', '.join)
df_new = result.reset_index()
df_new.to_csv('data/toronto-postalcode.csv')
print(df_new.shape)
df_new.head(15)

(103, 3)


,Postalcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Queen's Park
5,M9A,Downtown Toronto,Queen's Park
6,M1B,Scarborough,"Rouge, Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens, Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson, Garden District"


In [60]:
not_assigned_boroughs = df_new.index[df_new['Borough'] == 'Not assigned']
not_assigned_neighbourhoods = df_new.index[df_new['Neighbourhood'] == 'Not assigned']
not_assigned_neighbourhoods_and_borough = not_assigned_boroughs & not_assigned_neighbourhoods

print('The DataFrame shape is {}'.format(neighborhood_df.shape),'\n')
print('There are:')
print('  {} rows with Not assigned Borough'.format(not_assigned_boroughs.shape[0]))
print('  {} rows with Not assigned Neighbourhood'.format(not_assigned_neighbourhoods.shape[0]))
print('  {} rows with Not assigned Neighbourhood and Borough'.format(not_assigned_neighbourhoods_and_borough.shape[0]),'\n')


The DataFrame shape is (103, 3) 

There are:
  0 rows with Not assigned Borough
  0 rows with Not assigned Neighbourhood
  0 rows with Not assigned Neighbourhood and Borough 



In [62]:
print('After cleaning the DataFrame, its new shape is {}'.format(df_new.shape),'\n')
print('There are:')
print('  {} Postal codes'.format(df_new['Postalcode'].unique().shape[0]))
print('  {} Boroughs'.format(df_new['Borough'].unique().shape[0]))
print('  {} Neighbourhoods'.format(df_new['Neighbourhood'].unique().shape[0]))

After cleaning the DataFrame, its new shape is (103, 3) 

There are:
  103 Postal codes
  11 Boroughs
  102 Neighbourhoods
